In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical

# Param

In [2]:
data_dir = 'dataset/edge_detection'
classes = ['1000', '2000', '5000', '10000', '20000', '50000', '100000']
IMG_SIZE = (256,256)

EPOCHS = 15
BATCH_SIZE = 32

# Preprocessing

In [6]:
raw_dir = 'dataset/raw'

# Create output directory structure
Path(data_dir).mkdir(parents=True, exist_ok=True)
for cls in classes:
    Path(os.path.join(data_dir, cls)).mkdir(parents=True, exist_ok=True)

def apply_edge_detection(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.GaussianBlur(gray, (13, 13), 0)

    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, dx=1, dy=0, ksize=3)  # x
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, dx=0, dy=1, ksize=3)  # y
    
    # Combine the two gradients
    edge = cv2.magnitude(sobel_x, sobel_y)
    return edge

# Loop through each class
for cls in classes:
    raw_class_dir = os.path.join(raw_dir, cls)
    output_class_dir = os.path.join(data_dir, cls)
    
    # Process each image in the class folder
    for img_name in tqdm(os.listdir(raw_class_dir), desc=f'Processing {cls}'):
        img_path = os.path.join(raw_class_dir, img_name)
        
        # Read the image
        image = cv2.imread(img_path)

        # Apply edge detection
        edges = apply_edge_detection(image)

        # Save the processed image to the edge_detection directory
        output_img_path = os.path.join(output_class_dir, img_name)
        cv2.imwrite(output_img_path, edges)

print("Edge detection preprocessing completed.")

Processing 100000: 100%|██████████| 80/80 [00:03<00:00, 24.63it/s]

Edge detection preprocessing completed.


# Funcs

In [ ]:
def pad_descriptors(descriptors, max_length=128):
    if descriptors is None:
        return np.zeros((max_length, 128))  # Assuming SIFT has 128 features per keypoint
    if descriptors.shape[0] < max_length:
        # Pad with zeros
        padded = np.zeros((max_length, descriptors.shape[1]))
        padded[:descriptors.shape[0], :] = descriptors
        return padded
    else:
        # Truncate if there are more descriptors than max_length
        return descriptors[:max_length, :]
    

def extract_features(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, IMG_SIZE)

    sift = cv2.SIFT_create()
    orb = cv2.ORB_create()

    sift_keypoints, sift_descriptors = sift.detectAndCompute(image, None)
    orb_keypoints, orb_descriptors = orb.detectAndCompute(image, None)
    sift_descriptors = pad_descriptors(sift_descriptors)


    return sift_descriptors, orb_descriptors

    # if sift_descriptors is not None and orb_descriptors is not None:
    #     orb_descriptors_padded = np.pad(orb_descriptors, ((0, 0), (0, 128 - orb_descriptors.shape[1])), mode='constant', constant_values=0)
    #     combined_descriptors = np.vstack((sift_descriptors, orb_descriptors_padded))

    # return combined_descriptors

def load_dataset():
    x_sift = []
    x_orb = []
    y = []
    for label, cls in enumerate(classes):
        cls_dir = os.path.join(data_dir, cls)
        for img_name in os.listdir(cls_dir):
            img_path = os.path.join(cls_dir, img_name)
            sift_descriptor, orb_descriptor = extract_features(img_path)
            x_sift.append(sift_descriptor)  # Flatten descriptors
            y.append(label)
    print(x_sift)
    x_sift = np.array(x_sift)
    y = np.array(y)

    # Normalize feature vectors
    x_sift = x_sift / 255.0

    return x_sift, y

In [8]:
x, y = load_dataset()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=len(classes))
y_test = to_categorical(y_test, num_classes=len(classes))

[array([[32.,  4.,  0., ..., 35.,  1.,  4.],
       [ 1.,  2., 80., ...,  0.,  0.,  1.],
       [24.,  4.,  0., ...,  0.,  0.,  0.],
       ...,
       [12.,  0.,  0., ..., 67., 16.,  0.],
       [32.,  8., 18., ...,  0.,  0.,  1.],
       [27.,  1.,  1., ...,  0.,  0.,  1.]], dtype=float32), array([[25., 15., 77., ...,  0.,  0.,  0.],
       [ 8.,  1.,  0., ...,  4.,  7., 18.],
       [ 2.,  1.,  1., ...,  1.,  7., 13.],
       ...,
       [ 0.,  0.,  1., ...,  1.,  0.,  0.],
       [31.,  5.,  6., ...,  5.,  1.,  6.],
       [ 2.,  2.,  1., ...,  4., 16.,  3.]], dtype=float32), array([[54., 12.,  0., ..., 12.,  0.,  0.],
       [47.,  3.,  1., ...,  4.,  0.,  0.],
       [24.,  3.,  2., ...,  2.,  0.,  0.],
       ...,
       [52.,  3.,  0., ...,  1.,  0.,  3.],
       [21., 26.,  1., ..., 64., 46.,  2.],
       [ 9.,  2.,  4., ...,  0.,  0.,  0.]], dtype=float32), array([[39.,  0.,  0., ...,  0.,  1.,  1.],
       [ 0.,  0.,  0., ..., 62.,  0.,  0.],
       [ 4.,  0.,  0., ..., 16.,

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (560,) + inhomogeneous part.

In [ ]:
model = Sequential([
    Input(shape=(x.shape[1],)),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    x_train, 
    y_train, 
    validation_data=(x_test, y_test), 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE)

In [ ]:
# fungsi buat evaluate model
def modelEvaluation(history, m):    
    plt.figure(figsize=(14, 4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [ ]:
val_loss, val_accuracy = model.evaluate(val)

print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

modelEvaluation(history, model)